#This was the fix for dca:

mamba create -n enimpute python==3.8

conda activate enimpute 

/home/subudhak/miniconda3/envs/enimpute/bin/pip3 install dca 'keras>=2.4,<2.6' 'tensorflow>=2.0,<2.5'

https://github.com/theislab/dca/issues/66


In [1]:
import scanpy as sc
# from magic import MAGIC
import scanpy.external as sce


adata = sc.read_h5ad("/home/sah2p/ondemand/singlecell_data/SRP173393.h5ad")

In [2]:
adata
import numpy as np

# Check if the data is normalized
mean_expression = np.mean(adata.X)
print(f"Mean expression value: {mean_expression:.2f}")

Mean expression value: 0.05


In [7]:
if adata.raw is not None:
    print("Using raw count data from adata.raw.X")
adata.X = adata.raw.X.copy()
    
# Remove all-zero genes
sc.pp.filter_genes(adata, min_counts=1)


Using raw count data from adata.raw.X


In [9]:
import tensorflow as tf

# Enable TensorFlow 2.x behavior and disable session-based execution
tf.compat.v1.disable_eager_execution()

# Reset Keras session to avoid variable conflicts
tf.keras.backend.clear_session()

In [ ]:
sce.pp.dca(
    adata,
    copy=True,          # Whether to return a copy instead of modifying in place
    hidden_size=[64, 32, 64],  # Neural network architecture
    epochs=10,         # Number of training epochs
    batch_size=32,      # Batch size for training
    learning_rate=0.001, # Learning rate for optimization
    verbose=True        # Print progress messages
)
# sce.pp.dca(adata)

dca: Successfully preprocessed 34017 genes and 13252 cells.


2025-02-04 10:19:48.152264: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set


Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
count (InputLayer)              [(None, 34017)]      0                                            
__________________________________________________________________________________________________
enc0 (Dense)                    (None, 64)           2177152     count[0][0]                      
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 64)           192         enc0[0][0]                       
__________________________________________________________________________________________________
enc0_act (Activation)           (None, 64)           0           batch_normalization[0][0]        
____________________________________________________________________________________________

/home/sah2p/ondemand/tools/miniconda3/envs/dcaimpute/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:2323: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


11926/11926 [==============================] - 11s 947us/sample - loss: 0.2832 - val_loss: 0.1754
Epoch 2/10
11926/11926 [==============================] - 11s 925us/sample - loss: 0.2199 - val_loss: 0.1694
Epoch 3/10
11926/11926 [==============================] - 11s 927us/sample - loss: 0.2171 - val_loss: 0.1667
Epoch 4/10
11926/11926 [==============================] - 11s 935us/sample - loss: 0.2158 - val_loss: 0.1653
Epoch 5/10
11926/11926 [==============================] - 11s 922us/sample - loss: 0.2148 - val_loss: 0.1642
Epoch 6/10
11926/11926 [==============================] - 11s 914us/sample - loss: 0.2142 - val_loss: 0.1636
Epoch 7/10
11926/11926 [==============================] - 11s 902us/sample - loss: 0.2138 - val_loss: 0.1632
Epoch 8/10
11926/11926 [==============================] - 11s 897us/sample - loss: 0.2134 - val_loss: 0.1631
Epoch 9/10
11926/11926 [==============================] - 11s 904us/sample - loss: 0.2131 - val_loss: 0.1629
Epoch 10/10
11926/11926 [=====

In [12]:
adata.obs

,Orig.ident,nCount_RNA,nFeature_RNA,Percent.mt,Seurat_clusters,Celltype,Dataset,Tissue,Organ,Condition,Genotype,Libraries,ACE,dca_split,n_counts,size_factors
SRX5128615@@_AAACCTGAGAGTAAGG-1,SRX5128615,1131.0,897,0.000000,0,Root epidermis,SRP173393,Root tip,Root,Normal,Col-0,10x Genomics,6 days old seedling,train,1131.0,0.311656
SRX5128615@@_AAACCTGAGATCCGAG-1,SRX5128615,4346.0,1958,0.000000,23,Root endodermis,SRP173393,Root tip,Root,Normal,Col-0,10x Genomics,6 days old seedling,train,4346.0,1.197575
SRX5128615@@_AAACCTGAGCGCCTCA-1,SRX5128615,6849.0,2614,0.000000,16,G2/M phase,SRP173393,Root tip,Root,Normal,Col-0,10x Genomics,6 days old seedling,train,6849.0,1.887297
SRX5128615@@_AAACCTGAGGAATGGA-1,SRX5128615,2470.0,1524,0.000000,13,Root epidermis,SRP173393,Root tip,Root,Normal,Col-0,10x Genomics,6 days old seedling,train,2470.0,0.680628
SRX5128615@@_AAACCTGCACTTAACG-1,SRX5128615,12600.0,3711,0.007937,12,Non-hair,SRP173393,Root tip,Root,Normal,Col-0,10x Genomics,6 days old seedling,train,12600.0,3.472031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SRX5128617@@_TTTGGTTGTTACGCGC-1,SRX5128617,1860.0,1041,0.000000,1,Root cap,SRP173393,Root tip,Root,Normal,Col-0 (shr-3),10x Genomics,6 days old seedling,train,1860.0,0.512538
SRX5128617@@_TTTGTCAGTATGAATG-1,SRX5128617,3580.0,1290,0.000000,22,Unknow,SRP173393,Root tip,Root,Normal,Col-0 (shr-3),10x Genomics,6 days old seedling,train,3580.0,0.986498
SRX5128617@@_TTTGTCAGTGCGATAG-1,SRX5128617,12396.0,2438,0.000000,2,Lateral root cap,SRP173393,Root tip,Root,Normal,Col-0 (shr-3),10x Genomics,6 days old seedling,train,12396.0,3.415817
SRX5128617@@_TTTGTCATCACGCGGT-1,SRX5128617,3194.0,1329,0.000000,22,Unknow,SRP173393,Root tip,Root,Normal,Col-0 (shr-3),10x Genomics,6 days old seedling,train,3194.0,0.880132


In [55]:
adata.var.index.name = 'genes'
adata.obs.index.name = 'cells'
adata.raw.var.index.name = 'genes'

# adata.var.reset_index()

In [11]:
import tensorflow as tf

# Enable TensorFlow 2.x behavior and disable session-based execution
tf.compat.v1.disable_eager_execution()

# Reset Keras session to avoid variable conflicts
tf.keras.backend.clear_session()

In [57]:
if adata.raw is not None and '_index' in adata.raw.var.columns:
    print("Dropping `_index` from adata.raw.var")
    adata.raw.var = adata.raw.var.drop(columns=['_index'])

Dropping `_index` from adata.raw.var


AttributeError: can't set attribute

In [54]:
print("Checking problematic column in adata.var:", "_index" in adata.var.columns)
print("Checking problematic column in adata.raw.var:", "_index" in adata.raw.var.columns if adata.raw else "adata.raw is None")
print("adata.var columns:", adata.var.columns)
print("adata.raw.var columns:", adata.raw.var.columns if adata.raw else "No raw.var")

Checking problematic column in adata.var: False
Checking problematic column in adata.raw.var: True
adata.var columns: Index(['features'], dtype='object')
adata.raw.var columns: Index(['_index'], dtype='object')


In [61]:
adata.raw.var.columns = ['genes']

In [62]:
# magic_op = MAGIC()
# adata.layers["MAGIC"] = magic_op.fit_transform(adata.X)

# Save the MAGIC-imputed dataset
adata.write_h5ad("/home/sah2p/ondemand/singlecell_data/SRP173393magic_d.h5ad")
